<a href="https://colab.research.google.com/github/NikitaPugach/PythonToolkit/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Підсумкове завдання Пугач Микита МФ-41**

Розглядається граматика регулярних виразів
S -> S '|' S | A
A -> A ',' A | P
P -> P '*' | 'chr' | 'nil'

1) Виконуємо умові, що дозволять робиті парсинг ліво-рекурсивним спуском

  1.1) Пошук та видалення сторонніх нетерміналів

  1.1.1) Знайдемо всі непродуктивні нетермінали:


In [ ]:
    def find_unproductive_non_terminal(self):
        alive = []
        # Нахождение первого элемента, который войдет в alive
        for rules_for_symbols in self.grammar.rules:
            right_part = self.grammar.rules[rules_for_symbols]
            is_terminal = True
            # for rule in right_part:
            if isinstance(right_part, list):
                for symbol in right_part:
                    if isinstance(symbol, list):
                        for s in symbol:
                            is_terminal = s in self.grammar.E  # Проверка все ли символы в правой части терминалы
                            if not is_terminal:
                                break
                        if is_terminal:  # если да, то добавляем левую часть в alive
                            alive.append(rules_for_symbols)
                            break
                    else:
                        is_terminal = symbol in self.grammar.E  # Проверка все ли символы в правой части терминалы
                        if not is_terminal:
                            break
                if is_terminal:  # если да, то добавляем левую часть в alive
                    alive.append(rules_for_symbols)
                    break
            else:
                is_terminal = right_part in self.grammar.E  # Проверка все ли символы в правой части терминалы

                if is_terminal:  # если да, то добавляем левую часть в alive
                    alive.append(rules_for_symbols)
                    break

        # Основной алгоритм
        expansion = True

        while expansion:
            alive_size = len(alive)
            for rules_for_symbols in self.grammar.rules:
                right_part = self.grammar.rules[rules_for_symbols]
                # for rule in right_part:
                is_terminal_or_from_alive = True
                if isinstance(right_part, list):
                    for symbol in right_part:

                        if isinstance(symbol, list):
                            for s in symbol:
                                is_terminal_or_from_alive = s in self.grammar.E or s in alive  # Проверка все лисимволы
                                if not is_terminal_or_from_alive:  # в правой части терминалы либо из alive
                                    break
                            if is_terminal_or_from_alive:  # если да, то добавляем левую часть в alive
                                if not rules_for_symbols in alive:
                                    alive.append(rules_for_symbols)
                        else:
                            is_terminal_or_from_alive = symbol in self.grammar.E or symbol in alive  # Проверка все ли
                            if not is_terminal_or_from_alive:  # символы в правой части терминалы либо из alive
                                break

                    if is_terminal_or_from_alive:  # если да, то добавляем левую часть в alive
                        if not rules_for_symbols in alive:
                            alive.append(rules_for_symbols)

                else:
                    is_terminal_or_from_alive = right_part in self.grammar.E or right_part in alive
                    if is_terminal_or_from_alive:
                        if not rules_for_symbols in alive:
                            alive.append(rules_for_symbols)

            if alive_size == len(alive):  # Проверка расширилось ли alive
                expansion = False

        return alive

1.1.2) Знайдемо всі недосяжні нетермінали:

In [ ]:
    def find_unattainable_non_terminal(self):
        reachable = [self.grammar.S]

        expansion = True
        while expansion:
            reachable_size = len(reachable)

            for rules_for_symbols in self.grammar.rules:
                if rules_for_symbols in reachable:
                    right_part = self.grammar.rules[rules_for_symbols]
                    if isinstance(right_part, list):
                        for symbol in right_part:
                            if isinstance(symbol, list):
                                for s in symbol:
                                    if s in self.grammar.N:
                                        if not s in reachable:
                                            reachable.append(s)
                            else:
                                if symbol in self.grammar.N:
                                    if not symbol in reachable:
                                        reachable.append(symbol)
                    else:
                        if right_part in self.grammar.N:
                            if not right_part in reachable:
                                reachable.append(right_part)

            if len(reachable) == reachable_size:
                expansion = False

        return reachable

1.1.3) Видалимо спочатку всі непродуктивні, а потім недосяжні нетермінали:

In [ ]:
    def remove_all_rules_with_unproductive_non_terminal(self):
        alive = self.find_unproductive_non_terminal()

        to_delete = list(set(self.grammar.rules.keys()) - set(alive))

        # Удаляем все правила, в которых в левой части стоят недосягаемые нетерминалы
        [self.grammar.rules.pop(key) for key in to_delete]

        self.grammar.N = alive

        for rules_for_symbols in self.grammar.rules:
            right_part = self.grammar.rules[rules_for_symbols]

            if isinstance(right_part, list):
                for symbol in right_part:
                    if isinstance(symbol, list):
                        for s in symbol:
                            if not s in self.grammar.N:
                                if not s in self.grammar.E:
                                    self.grammar.rules.pop(s)
                    else:
                        if not symbol in self.grammar.N:
                            if not symbol in self.grammar.E:
                                self.grammar.rules.pop(symbol)
            else:
                if not right_part in self.grammar.N:
                    if not right_part in self.grammar.E:
                        self.grammar.rules.pop(right_part)

    def remove_all_rules_with_unattainable_non_terminal(self):
        reachable = self.find_unattainable_non_terminal()

        to_delete = list(set(self.grammar.rules.keys()) - set(reachable))

        # Удаляем все правила, в которых в левой части стоят недосягаемые нетерминалы
        [self.grammar.rules.pop(key) for key in to_delete]

        self.grammar.N = reachable

        for rules_for_symbols in self.grammar.rules:
            right_part = self.grammar.rules[rules_for_symbols]

            if isinstance(right_part, list):
                for symbol in right_part:
                    if isinstance(symbol, list):
                        for s in symbol:
                            if not s in self.grammar.N:
                                if not s in self.grammar.E:
                                    self.grammar.rules.pop(s)
                    else:
                        if not symbol in self.grammar.N:
                            if not symbol in self.grammar.E:
                                self.grammar.rules.pop(symbol)
            else:
                if not right_part in self.grammar.N:
                    if not right_part in self.grammar.E:
                        self.grammar.rules.pop(right_part)

    def remove_excess_non_terminals(self):
        self.remove_all_rules_with_unattainable_non_terminal()
        self.remove_all_rules_with_unproductive_non_terminal()

1.2) Знайдемо всі знникаючі нетермінали:

In [ ]:
 def find_vanishing_symbols(self):
        vanishing = []

        for rules_for_symbol in self.grammar.rules:
            if self.grammar.rules[rules_for_symbol] is None:
                vanishing.append(rules_for_symbol)

        expansion = True

        while expansion:
            vanishing_size = len(vanishing)

            for rules_for_symbol in self.grammar.rules:
                right_part = self.grammar.rules[rules_for_symbol]

                is_vanishing = False
                if isinstance(right_part, list):
                    for symbol in right_part:
                        if isinstance(symbol, list):
                            for s in symbol:
                                if s in vanishing:
                                    is_vanishing = True
                                else:
                                    is_vanishing = False
                                    break
                        else:
                            if symbol in vanishing:
                                is_vanishing = True
                            else:
                                is_vanishing = False
                                break
                    if is_vanishing is True:
                        if not rules_for_symbol in vanishing:
                            vanishing.append(rules_for_symbol)
                else:
                    if right_part in vanishing:
                        if not rules_for_symbol in vanishing:
                            vanishing.append(rules_for_symbol)

            if vanishing_size == len(vanishing):
                expansion = False

        return vanishing


1.3) Зробимо діагностування лівої рекурсії:

In [ ]:
    def create_graph(self):
        self.remove_all_rules_with_vanishing_symbols()

        array = [0] * len(self.grammar.N)
        for row in range(len(self.grammar.N)):
            array[row] = [0] * len(self.grammar.N)

        for rules_for_symbol in self.grammar.rules:
            right_part = self.grammar.rules[rules_for_symbol]

            if isinstance(right_part, list):
                for symbol in right_part:
                    if isinstance(symbol, list):
                        for s in symbol:
                            if s in self.grammar.N:
                                row = find_index(rules_for_symbol, self.grammar.N)
                                column = find_index(s, self.grammar.N)
                                array[row][column] = 1
                    else:
                        if symbol in self.grammar.N:
                            row = find_index(rules_for_symbol, self.grammar.N)
                            column = find_index(symbol, self.grammar.N)
                            array[row][column] = 1
                            print(self.grammar.N[row], end=" -> ")
                            print(self.grammar.N[column])

            else:
                if right_part in self.grammar.N:
                    row = find_index(rules_for_symbol, self.grammar.N)
                    column = find_index(right_part, self.grammar.N)
                    array[row][column] = 1

        return array

    def left_recursion_diagnosing(self):
        left_recursion_finder = LeftRecursionFinder(self)
        return left_recursion_finder.left_recursion_diagnosing()


class LeftRecursionFinder(object):
    def __init__(self, toolkit):
        self.toolkit = toolkit
        self.graph = toolkit.create_graph()
        self.current_stack = []
        self.recursion_elements = []

    def left_recursion_diagnosing(self):
        for i in range(self.graph):
            for j in range(self.graph[i]):
                if self.graph[i][j] is 1:
                    self.current_stack.append(self.toolkit.grammar.N[i])
                    self.recursion(j)
                if len(self.current_stack) > len(set(self.current_stack)):
                    self.recursion_elements.append(self.current_stack[0])
        return self.recursion_elements

    def recursion(self, elem):
        array = self.graph[elem]

        for i in range(len(array)):
            if array[i] is 1:
                self.current_stack.append(self.toolkit.grammar.N[elem])
                self.recursion(i)
